In [1]:
import os
print(f"Current working folder: {os.getcwd()}")

os.chdir('C:/Users/UHY6HC/Downloads/Thesis/RAG_LLM_DA')
print(f"Current working folder: {os.getcwd()}")

from stages.stage_1_learn_rules_from_data.data_loader import DataLoader
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from openai_llm.llm_init import LLM_Model
from utils import load_json_data, save_json_data, load_vectorstore_db, lookup_vector_db, calculate_similarity

Current working folder: c:\Users\UHY6HC\Downloads\Thesis\RAG_LLM_DA\notebooks
Current working folder: C:\Users\UHY6HC\Downloads\Thesis\RAG_LLM_DA


In [2]:
llm_instance = LLM_Model()

Loading data from config/llm_config.json


In [3]:
data_dict = load_vectorstore_db(llm_instance, 'icews14')
for collection in data_dict:
    print(f"{collection}: {len(data_dict[collection]['vector_db'].get()['documents'])} documents")

Loading data from config/data_embedding.json
facts: 181460 documents
rules: 0 documents


In [8]:


# filter = {"$and": [
# 	{"object": {"$in": ["Malaysia", 'China']}},
# 	{"subject": {"$in": ["Malaysia", 'China']}},
#     # {"relation": "Express_intent_to_cooperate"}
# ]
# }

filter = {"$or":[{"object": "Malaysia"}, {"subject": "Malaysia"}]}
# filter = {"subject": "Government_(United_States)"}
docs = lookup_vector_db("Express_intent_to_cooperate", filter, data_dict['facts']['vector_db'], llm_instance, top_k=100)
docs = sorted(docs, key=lambda x: x.metadata['timestamp_id'], reverse=True)
docs

[Document(metadata={'object': 'Thailand', 'object_id': 18, 'relation': 'inv_Express_intent_to_cooperate', 'relation_id': 243, 'subject': 'Malaysia', 'subject_id': 30, 'timestamp': '2014-12-04', 'timestamp_id': 338}, page_content='"Malaysia were the recipients of expressed intent to cooperate to/with Thailand on 2014-12-04"'),
 Document(metadata={'object': 'Malaysia', 'object_id': 30, 'relation': 'inv_Express_intent_to_cooperate', 'relation_id': 243, 'subject': 'Thailand', 'subject_id': 18, 'timestamp': '2014-12-04', 'timestamp_id': 338}, page_content='"Thailand were the recipients of expressed intent to cooperate to/with Malaysia on 2014-12-04"'),
 Document(metadata={'object': 'Thailand', 'object_id': 18, 'relation': 'Express_intent_to_cooperate', 'relation_id': 13, 'subject': 'Malaysia', 'subject_id': 30, 'timestamp': '2014-12-04', 'timestamp_id': 338}, page_content='"Malaysia expressed intent to cooperate to/with Thailand on 2014-12-04"'),
 Document(metadata={'object': 'Malaysia', 'o

---